# Markov Chains



The following Markov chain implementation is based on the Student MDP used in Lecture 2 of David Silver's lecture series.


<img src="/files/images/markov_chain.png", width=500px>


Notice that from any state we can see where we might transition to next. That is, notice that each state exhibits the **Markov property**: If we are currently at the Pub, we know which states we can go to next, and with what probability. Knowing that we were just in "Class 3" in the previous step doesn't tell us anything new about where we'll transition to next.

The associated **State Transition Matrix** for this Markov chain is merely copied from the states (Circles) and transition probabilities (Arrows) shown in the image:

In [ ]:
state_names = ["C1", "C2", "C3", "Pass", "Pub", "FB", "Sleep"]


# p_matrix[0][5] = Transition probability from "C1" to "FB"
# p_matrix[4][1] = Transition from "Pub" to "C2"
# etc.
p_matrix = [[0, .5, 0, 0, 0, .5, 0],
            [0, 0, .8, 0, 0, 0, .2],
            [0, 0, 0, .6, .4, 0, 0],
            [0, 0, 0, 0, 0, 0, 1],
            [.2, .4, .4, 0, 0, 0, 0],
            [.1, 0, 0, 0, 0, .9, 0],
            [0, 0, 0, 0, 0, 0, 1]]

If the state transition matrix is known, a trajectory can then be generated from the Markov chain: starting from any location in the chain a random path can be sampled using the transition probabilities to determine the next step. 

Once the agent reaches "Sleep" it will stop forever. This type of state is known as an absorbing state and can be thought of as a state with a self-transition probability of 1 (just like the Facebook state has a self-transition probability of 90%).

The existence of an absorbing state, and the fact that all other states have at least 1 path to it means that this Markov chain is actually an Absorbing Markov chain. Which for our purposes is just a fancy way of saying that any sample we take from the chain is guaranteed to end eventually. No infinite loops here.

In [33]:
from numpy.random import choice

class MarkovChain:
    def __init__(self, p_matrix, state_names, terminal_index=6):
        self.p_matrix = p_matrix
        self.state_names = state_names
        self.terminal_index = terminal_index
        
        row_lengths = [len(row) for row in p_matrix]
        assert (len(set(row_lengths)) <= 1), "p_matrix rows must have equal lengths!"
        assert len(p_matrix[0]) == len(p_matrix), "p_matrix must be square!"
    
    # Generates a random path through the chain
    def sample(self, start_state):
        path = []
        
        # Handle int or string as input state
        if isinstance(start_state, str):
            state_indx = self.state_names.index(start_state)
        else:
            state_indx = start_state
            
        state = state_indx
        
        while state != self.terminal_index:
            path.append(state)
            transition_p = self.p_matrix[state]
            
            # numpy.random.choice accepts choices (states 0 through len(p_matrix)) and
            # equally sized list of probabilities for those choices
            next_state = choice(range(len(self.p_matrix)), p=transition_p)
            state = next_state
        
        path.append(self.terminal_index)
        
        
        if isinstance(start_state, str):
            return self.pretty(path)
        else:
            return path
    
    def pretty(self, path):
        return [self.state_names[i] for i in path]
    
    # Uses the state_names to generate pretty paths instead of raw state integers

With our basic markov chain implementation we can sample trajectories at random, given a starting state.

In [34]:
chain = MarkovChain(p_matrix, state_names)
for i in range(10):
    print(chain.sample("C2"))
    print("\n --- \n")

['C2', 'C3', 'Pass', 'Sleep']

 --- 

['C2', 'C3', 'Pass', 'Sleep']

 --- 

['C2', 'C3', 'Pass', 'Sleep']

 --- 

['C2', 'Sleep']

 --- 

['C2', 'C3', 'Pass', 'Sleep']

 --- 

['C2', 'C3', 'Pub', 'C2', 'C3', 'Pass', 'Sleep']

 --- 

['C2', 'C3', 'Pass', 'Sleep']

 --- 

['C2', 'C3', 'Pub', 'C2', 'Sleep']

 --- 

['C2', 'C3', 'Pass', 'Sleep']

 --- 

['C2', 'Sleep']

 --- 

